In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

from src.dataset.chess_dataframe import ChessDataFrame, Sizes
from logging_config import setup_logging

# Setup logging
logger = setup_logging()

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# I need to make csv batches of data and uploads it to huggingface
dataset = ChessDataFrame(size=Sizes.extra_smol)
logger.info("Successfully initialized ChessDataset")

/Users/youcefboumar/PycharmProjects/ChessAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


10:45:58 - chessAI - INFO Using device: mps
10:45:58 - chessAI - INFO Initializing ChessDataset
10:45:58 - chessAI - INFO Repository Youcef/chessGames already exists
10:45:58 - chessAI - INFO Dataset already exists, loading...
10:46:00 - chessAI - INFO Successfully initialized ChessDataset


In [3]:
dataset.df_train.head()

,Result,WhiteElo,BlackElo,Player,Time,Eval,Raw Eval,Board,UCI,MovingPiece,CapturedPiece
29,1,1288,1287,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.0, 1.0, 1.0, 1.0, 0.9833333333333333, 1.0, ...","[0.15, 0.21, 0.11, 0.53, 0.17, 0.27, 0.25, 0.3...","[0.15, 0.21, 0.11, 0.53, 0.17, 0.27, 0.25, 0.3...",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,"[e2e4, e7e5, g1f3, d7d6, c2c3, f7f5, e4f5, c8f...","[P, p, N, p, P, p, P, b, P, p, N, p, P, n, P, ...","[None, None, None, None, None, None, p, P, Non..."
535,1,1844,1922,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.0, 1.0, 0.9833333333333333, 1.0, 0.96666666...","[0.15, 0.25, 0.22, 0.31, 0.21, 0.34, 0.37, 0.3...","[0.15, 0.25, 0.22, 0.31, 0.21, 0.34, 0.37, 0.3...",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,"[e2e4, c7c5, g1f3, b8c6, d2d4, c5d4, f3d4, g8f...","[P, p, N, n, P, p, N, n, N, p, N, p, B, p, N, ...","[None, None, None, None, None, P, p, None, Non..."
695,0,1816,1822,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.0, 1.0, 0.9833333333333333, 1.0, 0.96666666...","[-0.28, 0.33, -0.41, -0.41, -0.5, -0.47, -1.89...","[-0.28, 0.33, -0.41, -0.41, -0.5, -0.47, -1.89...",[rnbqkbnr/pppppppp/8/8/7P/8/PPPPPPP1/RNBQKBNR ...,"[h2h4, g7g6, g2g4, f8g7, f1g2, d7d5, d2d4, c7c...","[P, p, P, b, B, p, P, p, P, n, P, p, P, p, N, ...","[None, None, None, None, None, None, None, Non..."
557,1,1024,1111,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.0, 1.0, 0.9666666666666667, 1.0, 0.93333333...","[0.16, 0.23, -0.58, 0.96, -0.53, -0.56, -0.56,...","[0.16, 0.23, -0.58, 0.96, -0.53, -0.56, -0.56,...",[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...,"[d2d4, d7d5, e2e4, g8f6, b1c3, d5e4, f2f3, e4f...","[P, p, P, n, N, p, P, p, N, n, B, p, N, n, P, ...","[None, None, None, None, None, P, None, P, p, ..."
836,0,1350,1324,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.0, 1.0, 0.9833333333333333, 0.9666666666666...","[0.15, 0.21, -0.45, 0.16, 0.0, 2.14, -0.9, -0....","[0.15, 0.21, -0.45, 0.16, 0.0, 2.14, -0.9, -0....",[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,"[e2e4, e7e5, f2f4, f8c5, g1f3, g8h6, d2d4, e5d...","[P, p, P, b, N, n, P, p, N, q, P, k, B, n, P, ...","[None, None, None, None, None, None, None, P, ..."


In [4]:
# Let's examine the Eval column and find games with extreme evaluations
from chess import Board


extreme_evals = dataset.df_train[dataset.df_train['Eval'].apply(lambda x: any(abs(eval) >= 100 if isinstance(eval, (int, float)) else False for eval in x))]
print("Games with extreme evaluations (|eval| >= 100):")
print(extreme_evals[['Result', 'WhiteElo', 'BlackElo', 'Eval']].head())

# Find games with extreme evaluations and show the corresponding board state
boards = []
scores = []
for idx, row in extreme_evals.head().iterrows():
    evals = [(i, e) for i, e in enumerate(row['Eval']) if isinstance(e, (int, float))]
    if evals:
        # Find the index and value of the most extreme evaluation
        extreme_idx, extreme_eval = max(evals, key=lambda x: abs(x[1]))
        
        # Get the board state at that position
        board_states = row['Board']
        extreme_position = board_states[extreme_idx]
        
        print(f"\nGame {idx}:")
        print(f"Most extreme evaluation: {extreme_eval}")
        print(f"Position (FEN string) at extreme evaluation:")
        print(extreme_position)
        boards.append(Board(extreme_position))
        scores.append(extreme_eval)

Games with extreme evaluations (|eval| >= 100):
     Result  WhiteElo  BlackElo  \
29        1      1288      1287   
535       1      1844      1922   
695       0      1816      1822   
557       1      1024      1111   
836       0      1350      1324   

                                                  Eval  
29   [0.15, 0.21, 0.11, 0.53, 0.17, 0.27, 0.25, 0.3...  
535  [0.15, 0.25, 0.22, 0.31, 0.21, 0.34, 0.37, 0.3...  
695  [-0.28, 0.33, -0.41, -0.41, -0.5, -0.47, -1.89...  
557  [0.16, 0.23, -0.58, 0.96, -0.53, -0.56, -0.56,...  
836  [0.15, 0.21, -0.45, 0.16, 0.0, 2.14, -0.9, -0....  

Game 29:
Most extreme evaluation: 300.0
Position (FEN string) at extreme evaluation:
2r5/pp2b3/2k5/4BQpp/1n1P4/1NN5/PP5P/2K5 w - - 4 27

Game 535:
Most extreme evaluation: 310.0
Position (FEN string) at extreme evaluation:
5r1k/R5bp/3p4/1p6/4Q3/2P5/1P3qP1/3K3R w - - 0 30

Game 695:
Most extreme evaluation: -310.0
Position (FEN string) at extreme evaluation:
8/pp3kbQ/2p3p1/5r2/4r3/8/PPP5/1K3q1R w

In [5]:
dataset.df_train.iloc[0]

Result                                                           1
WhiteElo                                                      1288
BlackElo                                                      1287
Player           [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...
Time             [1.0, 1.0, 1.0, 1.0, 0.9833333333333333, 1.0, ...
Eval             [0.15, 0.21, 0.11, 0.53, 0.17, 0.27, 0.25, 0.3...
Raw Eval         [0.15, 0.21, 0.11, 0.53, 0.17, 0.27, 0.25, 0.3...
Board            [rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...
UCI              [e2e4, e7e5, g1f3, d7d6, c2c3, f7f5, e4f5, c8f...
MovingPiece      [P, p, N, p, P, p, P, b, P, p, N, p, P, n, P, ...
CapturedPiece    [None, None, None, None, None, None, p, P, Non...
Name: 29, dtype: object

In [7]:
# player (2), piece being moved (6), from square (64) to square (64)
# that's an input size of only 136 when hotshot encoded!

# capture (1), piece captured (6)
# so 143 with these extra features

PIECE_CHANNELS = {
    "P": 0,
    "N": 1,
    "B": 2,
    "R": 3,
    "Q": 4,
    "K": 5,
}

def encode_piece(pieces):
    """
    One-hot encode pieces. If piece is None, encode as [0,0,0,0,0,0].
    Returns a 2D array where each row is a one-hot vector.
    """
    num_pieces = len(PIECE_CHANNELS)
    encoded = np.zeros((len(pieces), num_pieces), dtype=np.int8)
    
    for i, piece in enumerate(pieces):
        if piece is not None:  # Only encode if piece exists
            channel = PIECE_CHANNELS[piece.upper()]
            encoded[i, channel] = 1
    
    return encoded

def aint(arr):
    return np.array([[int(x)] for x in arr], dtype=np.int8)

def encode_uci(uci):
    """
    Convert UCI move notation (e.g. 'e2e4') into a 128-length one-hot array
    representing the from and to squares (64 bits each)
    """
    # Chess square mapping: a1=0, b1=1, ..., h8=63
    file_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    
    # Calculate from_square index (0-63)
    from_file = file_map[uci[0]]
    from_rank = int(uci[1]) - 1
    from_square = from_rank * 8 + from_file
    
    # Calculate to_square index (0-63)
    to_file = file_map[uci[2]]
    to_rank = int(uci[3]) - 1
    to_square = to_rank * 8 + to_file
    
    # Create one-hot encoding
    encoded = np.zeros(128, dtype=np.int8)
    encoded[from_square] = 1        # First 64 bits for from_square
    encoded[to_square + 64] = 1     # Last 64 bits for to_square
    
    return encoded

def encode_ucis(ucis):
    return np.array([encode_uci(uci) for uci in ucis])

# let's preprocess the data!
players = dataset.df_train["Player"].apply(aint).to_numpy()
moving_pieces = dataset.df_train["MovingPiece"].apply(encode_piece).to_numpy()
captured_pieces = dataset.df_train["CapturedPiece"].apply(encode_piece).to_numpy()
uci_moves = dataset.df_train["UCI"].apply(encode_ucis).to_numpy()
results[0]

np.int64(1)

In [9]:
# Create a list to store all games, where each game is a list of move features
games = []

# Iterate through each game's data
for i in range(len(players)):
    games.append(np.concatenate([players[i], moving_pieces[i], captured_pieces[i], uci_moves[i]], axis=1))

# Convert the list of games into a numpy array
games

[array([[0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int8),
 array([[0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0]], dtype=int8),
 array([[0, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
 